# NFL Next Gen Stats - data download

Alejandro Ozymandias Cepeda Beltran

In [ ]:
import os
import subprocess
import zipfile
from pathlib import Path
import logging

# --- Configuración de logging para una salida clara ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def download_kaggle_competition_data(competition_slug: str, kaggle_json_path: str, download_dir: str = "data"):
    """
    Descarga y descomprime los datos de una competencia de Kaggle.

    Esta función realiza los siguientes pasos:
    2. Configura el token de API de Kaggle de forma segura.
    3. Descarga los archivos de la competencia especificada.
    4. Descomprime el archivo .zip en un directorio de destino.

    Args:
        competition_slug (str): El identificador de la competencia en Kaggle.
                                (p. ej., 'nfl-big-data-bowl-2026-prediction').
        kaggle_json_path (str): La ruta al archivo 'kaggle.json' que descargaste.
        download_dir (str): El nombre del directorio donde se guardarán los datos.
                            Por defecto es 'data'.

    Returns:
        bool: True si la operación fue exitosa, False en caso contrario.
    """
    try:
        # --- 2. Configurar el token de API ---
        kaggle_api_path = Path.home() / ".kaggle"
        kaggle_api_path.mkdir(parents=True, exist_ok=True)
        
        token_source = Path(kaggle_json_path)
        if not token_source.is_file():
            logging.error(f"El archivo del token 'kaggle.json' no se encontró en: {kaggle_json_path}")
            logging.error("Por favor, descárgalo desde tu cuenta de Kaggle y proporciona la ruta correcta.")
            return False
            
        token_dest = kaggle_api_path / "kaggle.json"
        token_dest.write_text(token_source.read_text())
        
        # Establecer permisos seguros (solo lectura/escritura para el propietario)
        os.chmod(token_dest, 0o600)
        logging.info("Token de API de Kaggle configurado de forma segura.")

        # --- 3. Descargar los datos ---
        logging.info(f"Descargando datos de la competencia: '{competition_slug}'...")
        # El comando de Kaggle descarga en el directorio de trabajo actual
        subprocess.run(
            ["kaggle", "competitions", "download", "-c", competition_slug],
            check=True
        )
        
        zip_filename = f"{competition_slug}.zip"
        if not Path(zip_filename).is_file():
            logging.error(f"La descarga falló. No se encontró el archivo '{zip_filename}'.")
            logging.error("Asegúrate de haber aceptado las reglas de la competencia en el sitio web de Kaggle.")
            return False

        # --- 4. Descomprimir los datos ---
        output_path = Path(download_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        
        logging.info(f"Descomprimiendo '{zip_filename}' en el directorio '{output_path}'...")
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(output_path)
        
        # Limpiar el archivo .zip después de la extracción
        Path(zip_filename).unlink()
        
        logging.info("¡Proceso completado! Los datos están listos en la carpeta '%s'.", output_path)
        return True

    except subprocess.CalledProcessError as e:
        logging.error(f"Falló un comando externo: {e}")
        logging.error("Revisa si el 'competition_slug' es correcto y si aceptaste las reglas de la competencia.")
        return False
    except Exception as e:
        logging.error(f"Ocurrió un error inesperado: {e}")
        return False

# --- Ejemplo de uso ---
if __name__ == "__main__":
    # Reemplaza con el identificador de la competencia que te interesa
    COMPETITION_NAME = "nfl-big-data-bowl-2026-prediction"
    
    # Reemplaza con la ruta donde guardaste tu archivo kaggle.json
    KAGGLE_TOKEN_FILE_PATH = "D:/codigos/kaggle/kaggle.json"

    download_kaggle_competition_data(
        competition_slug=COMPETITION_NAME,
        kaggle_json_path=KAGGLE_TOKEN_FILE_PATH
    )


2025-10-19 08:59:17,944 - INFO - Token de API de Kaggle configurado de forma segura.
2025-10-19 08:59:17,946 - INFO - Descargando datos de la competencia: 'nfl-big-data-bowl-2026-prediction'...


2025-10-19 08:59:24,004 - INFO - Descomprimiendo 'nfl-big-data-bowl-2026-prediction.zip' en el directorio 'data'...
2025-10-19 08:59:28,776 - INFO - ¡Proceso completado! Los datos están listos en la carpeta 'data'.


: 